In [1]:
#!pip install -r requirements.txt

In [2]:
import pandas as pd
from pycaret.regression import setup, get_config


In [3]:
# Step 1: Load the Excel data
df = pd.read_excel("data/ISBSG2016R1.1 - FormattedForCSV.xlsx")  # Replace with your file name

# Preview data
print(df.head())


   ISBSG Project ID External (EEF) - Data Quality Rating  \
0             10003                                    B   
1             10011                                    B   
2             10012                                    B   
3             10014                                    B   
4             10015                                    B   

   Project (PRF) - Year of Project External (EEF) - Industry Sector  \
0                             2015                    Communication   
1                             1996                     Construction   
2                             2002               Wholesale & Retail   
3                             2004                              NaN   
4                             2000               Wholesale & Retail   

  External (EEF) - Organisation Type Project (PRF) - Application Group  \
0                Telecommunications;              Business Application   
1                      Construction;              Business Appli

In [4]:
# Step 2: Initialise PyCaret setup for preprocessing


# the DataFrame is 'df' and target column is numeric
s = setup(
    data=df,
    target='Project (PRF) - Normalised Work Effort',
    preprocess=True,
    imputation_type='simple',  # mean/median imputation for numeric missing values
    numeric_imputation='mean',
    categorical_imputation='mode',
    profile=True,              # generates an automated profiling report
    session_id=123,
    verbose=True
)

# After setup, get the preprocessed dataset
processed_data = get_config('dataset')
print(processed_data.head())


,Description,Value
0,Session id,123
1,Target,Project (PRF) - Normalised Work Effort
2,Target type,Regression
3,Original data shape,"(7058, 52)"
4,Transformed data shape,"(7058, 160)"
5,Transformed train set shape,"(4940, 160)"
6,Transformed test set shape,"(2118, 160)"
7,Numeric features,26
8,Categorical features,25
9,Rows with missing values,100.0%


Loading profile... Please Wait!


      ISBSG Project ID External (EEF) - Data Quality Rating  \
6995             32577                                    A   
5294             26924                                    B   
2673             18663                                    B   
463              11468                                    B   
3520             21338                                    B   

      Project (PRF) - Year of Project External (EEF) - Industry Sector  \
6995                             2000                    Communication   
5294                             2006                        Insurance   
2673                             1998                              NaN   
463                              2002                          Banking   
3520                             1998                    Communication   

     External (EEF) - Organisation Type Project (PRF) - Application Group  \
6995                    Communications;              Business Application   
5294                  

In [6]:
# Save to CSV
processed_data.to_csv("data/preprocessed_data.csv", index=False)

In [5]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Dataset Profiling Report")
profile.to_file("data_profile.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


%|          | 0/52 [00:00<?, ?it/s]
%|▏         | 1/52 [00:00<00:09,  5.19it/s]
%|▊         | 4/52 [00:00<00:06,  6.95it/s]
%|██        | 11/52 [00:00<00:02, 18.72it/s]
%|████▏     | 22/52 [00:01<00:01, 26.20it/s]
%|████▊     | 25/52 [00:01<00:01, 17.62it/s]
100%|██████████| 52/52 [00:01<00:00, 31.79it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
print(df['Project (PRF) - Normalised Work Effort'].nunique())
print(df['Project (PRF) - Normalised Work Effort'].value_counts())


4214
Project (PRF) - Normalised Work Effort
995     9
304     9
620     9
473     9
62      9
       ..
1030    1
1243    1
4213    1
2225    1
80      1
Name: count, Length: 4214, dtype: int64


In [23]:
import seaborn as sns
import matplotlib.pyplot as plt

df = get_config("dataset")
corr = df.corr()

# Plot heatmap of correlations
plt.figure(figsize=(10, 8))
sns.heatmap(corr[[get_config('target')]].sort_values(by=get_config('target'), ascending=False),
            annot=True, cmap='coolwarm')
plt.title('Correlation with Target')
plt.show()


ValueError: could not convert string to float: 'B'

In [12]:
# Step 3 : Get the preprocessed dataset
# preprocessed_df = pull()  # Pulls the setup data summary
processed_data = get_config('X_train')  # Or get_config('dataset') for full

# Step 6 (Optional): Save the processed data
processed_data.to_csv("data/preprocessed_data.csv", index=False)

TypeError: 'NoneType' object is not subscriptable